# Deploy Web App on Azure Container Services (AKS)


In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored. 

The process is split into the following steps:
- Define our resource names
- Login to Azure
- Create resource group and create AKS
- Connect to AKS
- Deploy our app

We assume that this notebook is running on Linux and Azure CLI is installed before proceeding.

In [1]:
import os
import json
from testing_utilities import write_json_to_file
%load_ext dotenv

## Setup

Below are the various name definitions for the resources needed to setup AKS.

In [2]:
%%writefile --append .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# If you have multiple subscriptions select the subscription you want to use 
selected_subscription = "YOUR_SUBSCRIPTION"

# Resource group, name and location for AKS cluster.
resource_group = "RESOURCE_GROUP" 
aks_name = "AKS_CLUSTER_NAME"
location = "eastus"

In [3]:
%dotenv
image_name = os.getenv('docker_login') + os.getenv('image_repo')

## Azure account login

The command below will initiate a login to your Azure account. It will pop up with an url to go to where you will enter a one off code and log into your Azure account using your browser.

In [ ]:
!az login -o table

In [ ]:
!az account set --subscription "$selected_subscription"

In [ ]:
!az account show

You will also need to register the container service resources on your subscription if you haven't already done so.

In [ ]:
!az provider register -n Microsoft.ContainerService

In [ ]:
!az provider show -n Microsoft.ContainerService

## Create resources and dependencies

### Create resource group and AKS cluster

Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [ ]:
!az group create --name $resource_group --location $location

Below, we create the AKS cluster in the resource group we created earlier. This could take up to 15 minutes.

In [ ]:
%%time
!az aks create --resource-group $resource_group --name $aks_name --node-count 1 --generate-ssh-keys -s Standard_NC6

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [5]:
!sudo env "PATH=$PATH" az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [6]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "fbAKSClustergpu" as current context in /home/fboylu/.kube/config


Let's verify connection by listing the nodes.

In [7]:
!kubectl get nodes

NAME                       STATUS    ROLES     AGE       VERSION
aks-nodepool1-28016997-0   Ready     agent     60d       v1.9.6


Let's check the pods on our cluster.

In [8]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
kube-system   azureproxy-79c5db744-r5ggd              1/1       Running   2          60d
kube-system   heapster-55f855b47-4m7xr                2/2       Running   0          60d
kube-system   kube-dns-v20-7c556f89c5-4z4z6           3/3       Running   0          60d
kube-system   kube-dns-v20-7c556f89c5-mp5fh           3/3       Running   0          60d
kube-system   kube-proxy-k8t2c                        1/1       Running   0          60d
kube-system   kube-svc-redirect-z6ppp                 1/1       Running   8          60d
kube-system   kubernetes-dashboard-546f987686-8krxm   1/1       Running   2          60d
kube-system   tunnelfront-695bcbdc68-t4l8t            1/1       Running   34         60d


## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.

In [11]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": image_name,
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                              "mountPath":"/usr/local/nvidia",
                              "name": "nvidia",
                          }
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "nvidia",
                      "hostPath":{
                          "path":"/usr/local/nvidia"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [12]:
write_json_to_file(app_template, 'az-dl.json')

In [13]:
write_json_to_file(service_temp, 'az-dl.json', mode='a')

Let's check the manifest created.

In [14]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.5/lib"
                            }
                        ],
                        "image": "caia/kerastf-gpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
                 

Next, we will use kubectl create command to deploy our application.

In [15]:
!kubectl create -f az-dl.json

deployment.apps/azure-dl created
service/azure-dl created


Let's check if the pod is deployed. It may take as many as 10 minutes for the container to be ready.

In [17]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY     STATUS    RESTARTS   AGE
default       azure-dl-5f6b7dfb6f-kbxz6               1/1       Running   0          6m
kube-system   azureproxy-79c5db744-r5ggd              1/1       Running   2          60d
kube-system   heapster-55f855b47-4m7xr                2/2       Running   0          60d
kube-system   kube-dns-v20-7c556f89c5-4z4z6           3/3       Running   0          60d
kube-system   kube-dns-v20-7c556f89c5-mp5fh           3/3       Running   0          60d
kube-system   kube-proxy-k8t2c                        1/1       Running   0          60d
kube-system   kube-svc-redirect-z6ppp                 1/1       Running   8          60d
kube-system   kubernetes-dashboard-546f987686-8krxm   1/1       Running   2          60d
kube-system   tunnelfront-695bcbdc68-t4l8t            1/1       Running   34         60d


If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [18]:
!kubectl get events

LAST SEEN   FIRST SEEN   COUNT     NAME                                         KIND         SUBOBJECT                   TYPE      REASON                  SOURCE                              MESSAGE
46m         46m          1         aks-nodepool1-28016997-0.1548a985ff48b23b    Node                                     Normal    RegisteredNode          node-controller                     Node aks-nodepool1-28016997-0 event: Registered Node aks-nodepool1-28016997-0 in Controller
35m         35m          1         aks-nodepool1-28016997-0.1548aa2258f34dc5    Node                                     Normal    RegisteredNode          node-controller                     Node aks-nodepool1-28016997-0 event: Registered Node aks-nodepool1-28016997-0 in Controller
6m          6m           1         azure-dl-5f6b7dfb6f-kbxz6.1548abbc1c661966   Pod                                      Normal    Scheduled               default-scheduler                   Successfully assigned azure-dl-5f6b7dfb6f-kb

Check the logs for the application pod.

In [19]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))

In [20]:
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-08-07 17:58:41,382 CRIT Supervisor running as root (no user in config file)
2018-08-07 17:58:41,384 INFO supervisord started with pid 1
2018-08-07 17:58:42,387 INFO spawned: 'program_exit' with pid 9
2018-08-07 17:58:42,388 INFO spawned: 'nginx' with pid 10
2018-08-07 17:58:42,390 INFO spawned: 'gunicorn' with pid 11
2018-08-07 17:58:43,422 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
2018-08-07 17:58:44.007138: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2018-08-07 17:58:44.191739: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1392] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: ddde:00:00.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-08-07 17:58:44.191801: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1471] Adding visible gpu devices: 0
2018-08-07 

In [21]:
!kubectl get deployment

NAME       DESIRED   CURRENT   UP-TO-DATE   AVAILABLE   AGE
azure-dl   1         1         1            1           6m


It can take a few minutes for the service to populate the EXTERNAL-IP field below. This will be the IP you use to call the service. You can also specify an IP to use, please see the AKS documentation for further details.

In [22]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP   EXTERNAL-IP     PORT(S)        AGE
azure-dl   LoadBalancer   10.0.86.30   40.117.74.122   80:31341/TCP   6m


Next, we will [test our web application deployed on AKS](05_TestWebApp.ipynb).